# Experiment 3:

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/CA_Repo')
from gm_utils_v2 import gradient_matching, get_network, get_daparam, evaluate_synset, save_image, match_loss, get_dataset, train_and_test_model
from helpers import plot_gradient_loss_by_iteration_by_class_and_total

Mounted at /content/drive


In [ ]:
class Args:
    method = "DC"
    dataset = "MNIST"
    model =  "ConvNet"
    ipc = 1  # Images per class
    num_exp = 1
    num_eval = 3  # Number of evaluations per experiment
    Iteration = 25  # Training iterations
    lr_img = 0.1  # Learning rate for synthetic images
    lr_net = 0.01  # Learning rate for network
    batch_real = 256  # Batch size for real data
    batch_train = 128  # Batch size for synthetic data training
    init = "noise"  # Initialize synthetic data from noise
    hessian = True
    eval_mode = "S"
    data_path = "./data"
    save_path = "./results_t_3"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    epoch_eval_train = 50  # Number of epochs to train evaluation models
    outer_loop = 10  # Number of outer-loop updates for synthetic data\
    dis_metric = 'ours'

### LeNet

In [ ]:
args = Args()
eval_it_pool = [0, 5, 10, 15, 20, args.Iteration]
args.model = "LeNet"
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)

In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)

Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data from random noise...
[2024-12-16 22:13:35] Training begins
Iteration# 0
Evaluating model LeNet at iteration 0...
Image grid saved to ./results_t_3/vis_DC_MNIST_LeNet_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 214.7706
Iteration# 1
Iteration 1: Total Gradient Loss = 146.2020
Iteration# 2
Iteration 2: Total Gradient Loss = 133.6892
Iteration# 3
Iteration 3: Total Gradient Loss = 234.1001
Iteration# 4
Iteration 4: Total Gradient Loss = 251.6807
Iteration# 5
Evaluating model LeNet at iteration 5...
Image grid saved to ./results_t_3/vis_DC_MNIST_LeNet_1ipc_single_iter5.png
Iteration 5: Total Gradient Loss = 237.9265
Iteration# 6
Iteration 6: Total Gradient Loss = 115.4083
Iteration# 7
Iteration 7: Total Gradient Loss = 125.

In [ ]:
accs_all_exps

{'LeNet': [0.0367,
  0.0903,
  0.0942,
  0.0443,
  0.1396,
  0.1673,
  0.0544,
  0.0537,
  0.1193,
  0.0615,
  0.1195,
  0.0757,
  0.1393,
  0.0918,
  0.203,
  0.1724,
  0.1557,
  0.1791]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training LeNet on the synthetic data
Epoch 1/10, Loss: 2.3044
Epoch 2/10, Loss: 2.2888
Epoch 3/10, Loss: 2.2760
Epoch 4/10, Loss: 2.2623
Epoch 5/10, Loss: 2.2459
Epoch 6/10, Loss: 2.2277
Epoch 7/10, Loss: 2.2092
Epoch 8/10, Loss: 2.1869
Epoch 9/10, Loss: 2.1604
Epoch 10/10, Loss: 2.1311
Accuracy on test set (trained on synthetic data): 0.1189


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results_t_3/" + args.model + "_" )

Gradient loss by iteration plot for Class 0 saved to ./results_t_3/ConvNet__class0.png
Gradient loss by iteration plot for Class 1 saved to ./results_t_3/ConvNet__class1.png
Gradient loss by iteration plot for Class 2 saved to ./results_t_3/ConvNet__class2.png
Gradient loss by iteration plot for Class 3 saved to ./results_t_3/ConvNet__class3.png
Gradient loss by iteration plot for Class 4 saved to ./results_t_3/ConvNet__class4.png
Gradient loss by iteration plot for Class 5 saved to ./results_t_3/ConvNet__class5.png
Gradient loss by iteration plot for Class 6 saved to ./results_t_3/ConvNet__class6.png
Gradient loss by iteration plot for Class 7 saved to ./results_t_3/ConvNet__class7.png
Gradient loss by iteration plot for Class 8 saved to ./results_t_3/ConvNet__class8.png
Gradient loss by iteration plot for Class 9 saved to ./results_t_3/ConvNet__class9.png
Total gradient loss by iteration plot saved to ./results_t_3/ConvNet__total.png


### MLP

In [ ]:
args = Args()
args.model = "MLP"
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)

In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)


Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data from random noise...
[2024-12-16 22:18:18] Training begins
Iteration# 0
Evaluating model MLP at iteration 0...
Image grid saved to ./results_t_3/vis_DC_MNIST_MLP_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 3461.0446
Iteration# 1
Iteration 1: Total Gradient Loss = 3891.7101
Iteration# 2
Iteration 2: Total Gradient Loss = 1862.0416
Iteration# 3
Iteration 3: Total Gradient Loss = 2682.9709
Iteration# 4
Iteration 4: Total Gradient Loss = 2107.1576
Iteration# 5
Evaluating model MLP at iteration 5...
Image grid saved to ./results_t_3/vis_DC_MNIST_MLP_1ipc_single_iter5.png
Iteration 5: Total Gradient Loss = 1603.2299
Iteration# 6
Iteration 6: Total Gradient Loss = 1614.8220
Iteration# 7
Iteration 7: Total Gradient Loss = 1935.

In [ ]:
accs_all_exps

{'MLP': [0.0752,
  0.0757,
  0.0844,
  0.2353,
  0.1893,
  0.1065,
  0.2611,
  0.3041,
  0.3599,
  0.4699,
  0.5811,
  0.4805,
  0.5874,
  0.6524,
  0.6309,
  0.7465,
  0.6958,
  0.7275]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training MLP on the synthetic data
Epoch 1/10, Loss: 2.2962
Epoch 2/10, Loss: 2.2375
Epoch 3/10, Loss: 2.1803
Epoch 4/10, Loss: 2.1203
Epoch 5/10, Loss: 2.0521
Epoch 6/10, Loss: 1.9771
Epoch 7/10, Loss: 1.8945
Epoch 8/10, Loss: 1.8030
Epoch 9/10, Loss: 1.7043
Epoch 10/10, Loss: 1.6008
Accuracy on test set (trained on synthetic data): 0.6359


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results_t_3/" + args.model + "_" )

Gradient loss by iteration plot for Class 0 saved to ./results_t_3/ConvNet__class0.png
Gradient loss by iteration plot for Class 1 saved to ./results_t_3/ConvNet__class1.png
Gradient loss by iteration plot for Class 2 saved to ./results_t_3/ConvNet__class2.png
Gradient loss by iteration plot for Class 3 saved to ./results_t_3/ConvNet__class3.png
Gradient loss by iteration plot for Class 4 saved to ./results_t_3/ConvNet__class4.png
Gradient loss by iteration plot for Class 5 saved to ./results_t_3/ConvNet__class5.png
Gradient loss by iteration plot for Class 6 saved to ./results_t_3/ConvNet__class6.png
Gradient loss by iteration plot for Class 7 saved to ./results_t_3/ConvNet__class7.png
Gradient loss by iteration plot for Class 8 saved to ./results_t_3/ConvNet__class8.png
Gradient loss by iteration plot for Class 9 saved to ./results_t_3/ConvNet__class9.png
Total gradient loss by iteration plot saved to ./results_t_3/ConvNet__total.png


### CNN

In [ ]:
args = Args()
args.model = "ConvNet"
args.lr_img = 0.05
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)


In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)


Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data from random noise...
[2024-12-16 22:24:36] Training begins
Iteration# 0
Evaluating model ConvNet at iteration 0...
Image grid saved to ./results_t_3/vis_DC_MNIST_ConvNet_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 607673.9297
Iteration# 1
Iteration 1: Total Gradient Loss = 46510.7480
Iteration# 2
Iteration 2: Total Gradient Loss = 16329.3590
Iteration# 3
Iteration 3: Total Gradient Loss = 26338.8939
Iteration# 4
Iteration 4: Total Gradient Loss = 8783.7153
Iteration# 5
Evaluating model ConvNet at iteration 5...
Image grid saved to ./results_t_3/vis_DC_MNIST_ConvNet_1ipc_single_iter5.png
Iteration 5: Total Gradient Loss = 34127.2307
Iteration# 6
Iteration 6: Total Gradient Loss = -6674.3900
Iteration# 7
Iteration 7: Tota

In [ ]:
accs_all_exps

{'ConvNet': [0.099,
  0.0971,
  0.0875,
  0.6867,
  0.665,
  0.6115,
  0.7186,
  0.7112,
  0.7358,
  0.7235,
  0.6968,
  0.6919,
  0.7151,
  0.7391,
  0.7162,
  0.7237,
  0.7224,
  0.7735]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training ConvNet on the synthetic data
Epoch 1/10, Loss: 2.3806
Epoch 2/10, Loss: 1.9333
Epoch 3/10, Loss: 1.3856
Epoch 4/10, Loss: 1.0574
Epoch 5/10, Loss: 0.8122
Epoch 6/10, Loss: 0.5862
Epoch 7/10, Loss: 0.4248
Epoch 8/10, Loss: 0.3063
Epoch 9/10, Loss: 0.2199
Epoch 10/10, Loss: 0.1598
Accuracy on test set (trained on synthetic data): 0.8563


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results_t_3/" + args.model + "_" )

Gradient loss by iteration plot for Class 0 saved to ./results_t_3/ConvNet__class0.png
Gradient loss by iteration plot for Class 1 saved to ./results_t_3/ConvNet__class1.png
Gradient loss by iteration plot for Class 2 saved to ./results_t_3/ConvNet__class2.png
Gradient loss by iteration plot for Class 3 saved to ./results_t_3/ConvNet__class3.png
Gradient loss by iteration plot for Class 4 saved to ./results_t_3/ConvNet__class4.png
Gradient loss by iteration plot for Class 5 saved to ./results_t_3/ConvNet__class5.png
Gradient loss by iteration plot for Class 6 saved to ./results_t_3/ConvNet__class6.png
Gradient loss by iteration plot for Class 7 saved to ./results_t_3/ConvNet__class7.png
Gradient loss by iteration plot for Class 8 saved to ./results_t_3/ConvNet__class8.png
Gradient loss by iteration plot for Class 9 saved to ./results_t_3/ConvNet__class9.png
Total gradient loss by iteration plot saved to ./results_t_3/ConvNet__total.png
